# Extending SoS

SoS can be easily extended with new actions, targets, converters, file previewers, and support for new languages. To make the extension available to other users, you can either create and distribute a separate package, or extend SoS and send us a [pull request](https://help.github.com/articles/about-pull-requests/).

## Understanding `entry_points`

SoS makes extensive use of [**entry points**](http://setuptools.readthedocs.io/en/latest/setuptools.html#dynamic-discovery-of-services-and-plugins), which allows external modules to register their features in the file system to make them available to other modules. It can be confusing initially but [this stack overflow ticket](http://stackoverflow.com/questions/774824/explain-python-entry-points) explains it quite well.

To register additional feature with SoS, you either need to extend `entry_points` of the [`setup.py` script](https://github.com/vatlab/SOS/blob/master/setup.py) of SoS, or create your own package with these `entry_points`. Option `extras_require` can be used to specify additional dependent packages for these features. For example, you can create a package with the following entry_points to provide support for ruby.

```
    entry_points='''
[sos-language]
ruby = sos-ruby.kernel:sos_ruby

[sos-targets]
Ruby_Library = sos-ruby.target:Ruby-Library
'''
```

With the installation of this package, `sos` would be able to obtain a class `sos_ruby` from module `sos-ruby.kernel`, and use it to work with the `ruby` language.

## Defining your own actions

Under the hood an action is a normal Python function that is decorated as `SoS_Action`. The `decorator` defines the common interface of actions and calls the actual function. To define your own action, you generally need to 

```
from sos.actions import SoS_Action

@SoS_Action()
def my_action(*args, **kwargs):
    pass
```

The decorator accepts two optional parameters:

* `run_mode=['interactive', 'run']` meaning the action will be executable in batch and interactive (Jupyter kernel) mode, and not in `dryrun` mode. An action usually behave the same in both modes but can be tailered for the environment in which it runs. 
* `acceptable_args=['*']` which can be used to specify a list of acceptable parameter (`*` matches all keyword args). An exception will be raised if an action is defined with a list of `acceptable_args` and is called with an unrecognized argument.

You then need to add an entry to `entry_points` as

```
[sos-actions]
my_action = mypackage.mymodule:my_action
```

If the execution of action depends on some other targets, you can raise an `UnknownTarget` with the target so that the target can be obtained. For example, if your action depends on a particular `R_library`, you can test the existence of the target as follows:

```
from sos.target import UnknownTarget
from sos.R.target import R_library

@SoS_Action()
def my_action(script, *args, **kwargs):
    if not R_library('somelib').exists():
        raise UnknownTarget(R_library('somelib'))
    # ...
```

## Additional targets

Adding additional target is similar with a class derived from [`BaseTarget`](https://github.com/vatlab/SOS/blob/master/src/sos/target.py).

```
from sos.target import BaseTarget

class my_target(BaseTarget):
    def __init__(self, *args, **kwargs):
        pass
```

You will need to define several member functions for this class, most notably `exists` that checks the existence of the target. This function accept a parameter `mode`, which can be `target`, `signature`, or `any`. Like a file target, a target can be considered exist as long as its signature exists, and if the real target is not needed for the execution of the workflow.


The details of this class can be found at the source code of [`BaseTarget`](https://github.com/vatlab/SOS/blob/master/src/sos/target.py). The [`R_Library`](https://github.com/vatlab/SOS/blob/master/src/sos/R/target.py) provides a good example of a **virtual target** that do not have a fixed corresponding file, can be checked for existence, and actually attempts to obtain (install a R library) the target when it is checked.

After you defined your target, you can make it available to SOS by adding an appropriate entry point

```
[sos-targets]
my_target = mypackage.mymodule:my_target
```

## File format conversion

To convert between sos and another file format, you would need to define two function, one returnning an [`argparse.ArgumentParser`](https://docs.python.org/3/library/argparse.html) that parse converter arguments, and one performing real file conversion.

Suppose you would like to convert `.sos` to a `.xp` format, you can define these two functions as follows

```
import argparse
from sos.sos_script import SoS_Script

def get_my_converter_parser():
    parser = argparse.ArgumentParser('sos_xp')
    parser.add_argument('--theme', 
        help='Style of output format')
    return parser

def my_converter(source_file, dest_file, args=None, unknown_args=[]):
    # parse additional_args to obtain converter-specific options
    # then convert from source_file to dest_file
    script = SoS_Script(source_file)
    for section in script.sections:
        # do something

if __name__ == '__main__':
    parser = get_my_converter_parser()
    args, unknown_args = parser.parse_known_args(sys.argv[3:])
    my_converter(sys.argv[1], sys.argv[2], args, unknown_args)

```

You can then register the converter in `setup.py` as

```
[sos-converters]
fromExt-toExt.parser: mypackage.mymodule:get_my_converter_parser
fromExt-toExt.func: mypackage.mymodule:my_converter
```

Here `fromExt` is file extension without leading dot, `toExt` is destination file extension without leading dot, or a format specified by the `--to` parameter of command `sos convert`. If `dest_file` is unspecified, the output should be written to standard output.

This example uses `if __name__ == '__main__'` section so that the converter can be used as a standandalone program, which is not needed but a great way for testing purposes. Note that the input and output files are handled by `sos convert` so the parser only needs to parse converter-specific options.

## Preview additional formats

Adding a preview function is very simple. All you need to is to define a function that returns preview information, and add an entry point to link the function to certain file format.

More specifically, a previewer should be specified as

```
pattern,priority = preview_module:func
```

or

```
module:func,priority = preview_module:func
```

where

1. `pattern` is a pattern that matches incoming filename (see module fnmatch.fnmatch for details)
2. `module:func` specifies a function in module that detects the type of input file.
3. `priority` is an integer number that indicates the priority of previewer in case multiple pattern or function matches the  same file. Developers of third-party previewer can override an existing previewer by specifying a higher priority number.
4. `preview_module:func` points to a function in a module. The function should accept a filename as the only parameter, and  returns either

   * A string that will be displayed as plain text to standard output.
   * A dictionary that will be returned as `data` field of `display_data` (see [Jupyter documentation](http://jupyter-client.readthedocs.io/en/latest/messaging.html) for details). The dictionary typically has `text/html` for HTML output, "text/plain" for plain text, and "text/png" for image presentation of the file.

## Support for a language.

SoS needs to know a few things before it can support a language properly,

1. The Jupyter kernel this language uses to work with Jupyer, which is a `ir` kernel for language `R`.
2. How to translate a Python object to a **similar** object in this language
3. How to translate an object in this language to a **similar** object in Python.
4. The color of the prompt of cells executed by this language.
5. (Optional but recommend). Information of a running session.
6. Optional options for interacting with the language on frontend.

It is important to understand that, instead of providing object that is native to the **sender** language (e.g. use [`rpy2`](http://rpy2.bitbucket.org/) to wrap R objects in Python), SoS tries to provide object that is native to the **recipient** language. That is to say, although objects

```
a = 1
b = c(1, 2)
```

are of the same type `numeric` in R, they are translated to Python in different types

```
a = 1
b = [1, 2]
```

To support a new language, you will need to write a Python package that defines a class, say `mylanguage`, that should provide the following class attributes:

1. `supported_kernels`: a dictionary of language and names of the kernels that the language supports, such as `{'R': ['ir']}`. If multiple kernels are supported, SoS will look for a kernel with matched name in the order that is specified (e.g. `{'JavaScript': ['ijavascript', 'inodejs']}`). Multiple languages can be specified if a language module supports multiple languages (e.g. `Matlab` and `Octave`).
2. `background_color`: a name or `#XXXXXX` value for a color that will be used in the prompt area of cells that are executed by the subkernel. An empty string can be used for using default notebook color.
3. `options`: A Python dictionary with options that will be passed to the frontend. Currently two options `variable_pattern` and `assignment_pattern` are supported. Both options should be regular expressions in JS style. 
   * Option `variable_pattern` is used to identify if a statement is a simple variable (nothing else). If this option is defined and the input text (if executed at the side panel) matches the pattern, SoS will prepend `%preview` to the code. This option is useful only when `%preview var` displays more information than `var`.
   * Option `assignment_pattern` is used to identify if a statement is an assignment operation. If this option is defined and the input text matches the pattern, SoS will prepend `%preview var` to the code where `var` should be the first matched portion of the pattern (use `( )`). This mechanism allows SoS to automatically display result of an assignment when you step through the code.
   
An instance of the class would be initialized with the sos kernel and the name of the subkernel, which does not have to be one of the `supported_kernels` (could be self-defined) and should provide the following attributes and functions:

1. `init_statements`: a statement that will be executed by the sub-kernel when the kernel starts. This statement usually defines functions to convert object to Python.
2. `get_vars`: a Python function that transfer a Python variable to the subkernel.
3. `put_vars`: a Python function that put one or more variables in the subkernel to SoS or another subkernel.
4. `sessioninfo`: a Python function that returns information of the running kernel, usually including version of the language, the kernel, and currently used packages and their versions. For `R`, this means a call to `sessionInfo()` function. The return value of this function can be a string, a list of strings or `(key, value)` pairs, or a dictinary. The function will be called by the `%sessioninfo` magic of SoS.


### Obtain variable from SoS

The `get_vars` function should be defined as

```
def get_vars(self, var_names)
```
where 

* `self` is the language instance with access to the SoS kernel, and
* `var_names` are names in the sos dictionary.

This function is responsible for probing the type of Python variable and create a similar object in the subkernel.

For example, to create a Python object `b = [1, 2]` in `R` (magic `%get`), this function could

1. Obtain a R expression to create this variable (e.g. `b <- c(1, 2)`)
2. Execute the expression in the subkernel to create variable `b` in it.

Note that the function `get_vars` can change the variable name because a valid variable name in Python might not be a valid variable name in another language. The function should give a warning if this happens.

### Send variables to other kernels

The `put_vars` function should be defined as

```
def put_vars(self, var_names, to_kernel=None)
```
where

1. `self` is the language instance with access to the SoS kernel
2. `var_name` is a list of variables that should exist in the subkernel. Because a subkernel is responsible for sharing variables with names starting with `sos` to SoS automatically, this function should be called to pass these variables even when `var_names` is empty.
3. `to_kernel` is the destination kernel to which the variables should be passed.

Depending on destination kernel, this function can:

* If the destination kernel is `sos`, the function should return a dictionary of variables that will be merged to the SoS dictionary.
* If direct variable transfer is not supported by the language, the function can return a Python dictionary, in which case the language transfers the variables to SoS and let SoS pass along to the destination kernel.
* If direct variable transfer is supported, the function should return a string. SoS will evaluate the string in the destination kernel to pass variables directly to the destination kernel.
  
So basically, a language can start with an implementation of `put_vars(to_kernel='sos')` and let SoS handle the rest. If need arises, it can

* Implement variable exchanges between instances of the same language. This can be useful because there are usually lossness and more efficient methods in this case.
* Put variable to another languages where direct varable transfer is much more efficient than transferring through SoS.

For example, to send a `R` object `b <- c(1, 2)` from subkernel `R` to `SoS` (magic `%put`), this function can

1. Execute an statement in the subkernel to get the value(s) of variable(s) in some format, for example, a string `"{'b': [1, 2]}"`.
2. Post-process these varibles to return a dictionary to SoS.

The [`R` sos extension](https://github.com/vatlab/SOS/blob/master/src/sos/R/kernel.py) provides a good example to get you started.

**NOTE**: Unlike other language extension mechanisms in which the python module can get hold of the "engine" of the interpreter (e.g. `saspy` and matlab's Python extension start the interpreter for direct communication) or have access to lower level API of the language (e.g. `rpy2`), SoS only have access to the interface of the language and perform all conversions by executing commands in the subkernels and intercepting their response. Consequently,

1. Data exchange can be slower than other methods.
2. Data exchange is less dependent on version of the interpreter.
2. Data exchange can happen between a local and a remote kernel.

Also, although it can be more efficient to save large datasets to disk files and load in another kernel, this method does not work for kernels that do not share the same filesystem. We currently ignore this issue and assume all kernels have access to the same file system.

## Adding a subcommad (addon)

If you would like to add a complete subcommand as an addon to SoS, you will need to define two functions and add them to `setup.py` as two entry points, one with suffix `.args` and one with suffix `.func`.

```
[sos_addons]
myaddon.args = yourpackage.module:addon_parser
myaddon.func = yourpackage.module:addon_func
```

The `addon_parser` function should use module `argparse` to return an `ArgumentParser` object. SoS would obtain this parser and add it as a subparse of the SoS main parser so that the options can be parsed as

```
sos myaddon options
```

The `addon_func` should be defined as

```
def addon_func(args, unknown_args)
```

with `args` being the parsed known arguments, and `unknown_args` being a list of unknown arguments that you can process by youself.